In [33]:
import pandas as pd
import numpy as np

### Selecionando disciplinas de acordo com o id_componente filtrado em curriculos
No notebook 02, a seleção de disciplinas é feita passando os departamentos em uma lista, como filtramos 3 cursos, que é o caso do BCT, BTI e EngComp, era passado a lista com esses departamentos, porém, algumas disciplinas são ofertadas no DIMAP ou em outros departamentos, então foi pensado em filtrar pelo id_componente filtrados em currículo.

#### Problema da abordagem
Fazendo essa abordagem, algumas disciplinas antigas são filtradas, como é o caso de cálculo aplicado, que não é mais ofertado pela ECT, esse problema acontece porque a estrutura curricular que consta de EngComp (a única) é uma antiga, mesmo sendo de 2012 (mesmo ano da vingente).

In [34]:
disciplinas = pd.read_csv("../data/componentes-curriculares-presenciais-ok.csv")
disciplinas.shape

(36079, 30)

In [35]:
# departamentos_selecionados = ['ESCOLA DE CIÊNCIAS E TECNOLOGIA', 'INSTITUTO METROPOLE DIGITAL',
#                              'DEPARTAMENTO DE ENGENHARIA DE COMPUTACAO E AUTOMACAO', 'DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA']
# disc_sel = disciplinas[disciplinas.unidade_responsavel.isin(departamentos_selecionados)]
# disc_sel


In [36]:
cursos = pd.read_csv("../data/cursos-de-graduacao.csv", sep=";")
estr_curric = pd.read_csv("../data/estruturas-curriculares.csv", sep=";")
curric_comp = pd.read_csv("../data/curriculo-componente-graduacao.csv", sep=";")

In [37]:
dict_cursos =  {'CET' : 'CIÊNCIAS E TECNOLOGIA','BTI' : 'TECNOLOGIA DA INFORMAÇÃO','ENGCOMP' : 'ENGENHARIA DE COMPUTAÇÃO'}
cursos_selecionados = cursos[cursos.nome.isin(dict_cursos.values())]
cursos_selecionados[['nome','id_curso']]

,nome,id_curso
15,CIÊNCIAS E TECNOLOGIA,10320810
38,ENGENHARIA DE COMPUTAÇÃO,2000026
111,TECNOLOGIA DA INFORMAÇÃO,92127264


#### Estrutura curricular de engComp e BTI já estão as atuais de 2012 e 2014, respectivamente

In [38]:

estr_curric_sel = estr_curric[estr_curric.id_curso.isin(cursos_selecionados.id_curso.unique())]
estr_bct = estr_curric_sel.loc[estr_curric_sel.nome_curso.isin([dict_cursos['CET']]) & (estr_curric_sel.ano_entrada_vigor < 2016)].id_curriculo
estr_curric_sel = estr_curric_sel[~estr_curric_sel.id_curriculo.isin(estr_bct)]

In [63]:
estr_curric_sel[['id_curriculo','nome_matriz' ,'ano_entrada_vigor']]

,id_curriculo,nome_matriz,ano_entrada_vigor
21,101658777,TECNOLOGIA DA INFORMAÇÃO - NATAL - MT - BACHAR...,2014
22,101669382,TECNOLOGIA DA INFORMAÇÃO - NATAL - N - BACHARE...,2014
44,101659860,TECNOLOGIA DA INFORMAÇÃO - NATAL - N - BACHARE...,2014
47,101668583,TECNOLOGIA DA INFORMAÇÃO - NATAL - MT - BACHAR...,2014
115,105755388,TECNOLOGIA DA INFORMAÇÃO - NATAL - MT - BACHAR...,2014
136,115765126,CIÊNCIAS E TECNOLOGIA - NATAL - MT - BACHARELADO,2016
234,102200805,TECNOLOGIA DA INFORMAÇÃO - NATAL - MT - BACHAR...,2014
244,115305512,CIÊNCIAS E TECNOLOGIA - NATAL - MT - BACHARELADO,2016
254,89059693,ENGENHARIA DE COMPUTAÇÃO - NATAL - MT - FORMAÇÃO,2012
264,115306301,CIÊNCIAS E TECNOLOGIA - NATAL - N - BACHARELADO,2016


In [65]:
curric_comp = curric_comp[curric_comp.id_curriculo.isin(estr_curric_sel.id_curriculo)]
disc_sel = disciplinas[disciplinas.id_componente.isin(curric_comp.id_componente_curricular)]
disc = pd.merge(disc_sel, curric_comp,left_on='id_componente', right_on='id_componente_curricular', how='left')
disc = disc[['id_componente', 'codigo','nome','unidade_responsavel','id_curriculo','tipo_vinculo_componente']]

In [67]:
disc.to_csv("../outputs/disciplinas-selecionadas.csv", encoding='UTF-8', )
disc

,id_componente,codigo,nome,unidade_responsavel,id_curriculo,tipo_vinculo_componente
0,20029,DCA0414,INTRODUCAO A ROBOTICA,DEPARTAMENTO DE ENGENHARIA DE COMPUTACAO E AUT...,115802434,OPTATIVO
1,20029,DCA0414,INTRODUCAO A ROBOTICA,DEPARTAMENTO DE ENGENHARIA DE COMPUTACAO E AUT...,89059693,OPTATIVO
2,20030,DCA0441,TOPICOS ESPECIAIS EM ROBOTICA,DEPARTAMENTO DE ENGENHARIA DE COMPUTACAO E AUT...,89059693,OPTATIVO
3,20033,FIS0313,ONDAS E FISICA MODERNA,DEPARTAMENTO DE FÍSICA TEÓRICA E EXPERIMENTAL,101669220,OPTATIVO
4,20033,FIS0313,ONDAS E FISICA MODERNA,DEPARTAMENTO DE FÍSICA TEÓRICA E EXPERIMENTAL,101669382,OPTATIVO
5,20033,FIS0313,ONDAS E FISICA MODERNA,DEPARTAMENTO DE FÍSICA TEÓRICA E EXPERIMENTAL,101668583,OPTATIVO
6,20033,FIS0313,ONDAS E FISICA MODERNA,DEPARTAMENTO DE FÍSICA TEÓRICA E EXPERIMENTAL,101658777,OPTATIVO
7,20033,FIS0313,ONDAS E FISICA MODERNA,DEPARTAMENTO DE FÍSICA TEÓRICA E EXPERIMENTAL,102200805,OPTATIVO
8,20033,FIS0313,ONDAS E FISICA MODERNA,DEPARTAMENTO DE FÍSICA TEÓRICA E EXPERIMENTAL,105755388,OPTATIVO
9,20033,FIS0313,ONDAS E FISICA MODERNA,DEPARTAMENTO DE FÍSICA TEÓRICA E EXPERIMENTAL,101659860,OPTATIVO
